# **Installations & imports**

In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os 
os.environ['KAGGLE_USERNAME'] = "sarrabendaho"
os.environ['KAGGLE_KEY'] = "21d2fd8f032a7f87c75532412395faf4"

In [3]:
!kaggle competitions download -c jumia-purchase-prediction

  0% 0.00/518k [00:00<?, ?B/s]
100% 518k/518k [00:00<00:00, 69.1MB/s]


In [4]:
!unzip /content/jumia-purchase-prediction.zip

Archive:  /content/jumia-purchase-prediction.zip
  inflating: jum_test.csv            
  inflating: jum_train.csv           
  inflating: sample_submission.csv   


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

# **Data pre-processing**

In [6]:
train_df = pd.read_csv("/content/jum_train.csv")
test_df = pd.read_csv("/content/jum_test.csv")
train_df.head()


,Cust_id,id_purchase,CAT I,CAT II,revenue ( $ Algérien),city,platform,gender,Order date,month
0,153926,5774DZ8526,Accessoire femmes,Sacs,55251.0,A,Web,male,2022-07-19,7
1,153929,4224DZ8573,Accessoire femmes,Lunettes de soleil,444479.0,A,Web,male,2022-07-19,7
2,153935,7178DZ8657,Accessoire femmes,Lunettes de soleil,242911.0,A,Web,female,2022-07-19,7
3,153935,3598DZ8583,Accessoire femmes,Lunettes de soleil,267374.0,A,Web,female,2022-07-20,7
4,153950,5142DZ8612,Accessoire femmes,Sacs,102590.0,A,Web,female,2022-07-19,7


## **Feature engineering**

In [7]:
#encoding CAT II

cols = train_df["CAT II"].unique()
for item in cols:
  if not item in test_df["CAT II"].unique():
    print(item)


for item in train_df["CAT II"].unique():
  if not item in test_df["CAT II"].unique():
    test_df.loc[len(test_df)] = [15403	,'8904DZ8715'	,"Accessoire femmes" ,	 item	, 273851.0,	"A",	"Web",	"male"	, "2022-07-21", 7	]
  

for item in train_df["CAT I"].unique():
  if not item in test_df["CAT I"].unique():
    print(item)    


for item in train_df["CAT I"].unique():
  if not item in test_df["CAT I"].unique():
      test_df.loc[len(test_df)] = [15403	,'8904DZ8715'	, item ,	"Lunettes de soleil"	, 273851.0,	"A",	"Web",	"male"	, "2022-07-21", 7	]

In [ ]:
#perform one-hot encoding on 'CAT II' column 


#creating instance of one-hot-encoder
oh_cat1 = OneHotEncoder(handle_unknown='ignore')

oh_cat1.fit(train_df[['CAT II']])
encoded = oh_cat1.transform(train_df[['CAT II']]).toarray()
columns = oh_cat1.get_feature_names([''])
encoder_df = pd.DataFrame(encoded,columns=columns)
train_df = train_df.join(encoder_df)
train_df.drop(['CAT II'], axis=1, inplace=True)
train_df.head()

#test
encoded = oh_cat1.transform(test_df[['CAT II']]).toarray()
encoder_df = pd.DataFrame(encoded,columns=columns)
test_df = test_df.join(encoder_df)
test_df.drop(['CAT II'], axis=1, inplace=True)
test_df.head()

In [14]:
#dropping CAT I
train_df.drop(columns={'CAT I'}, inplace=True)
test_df.drop(columns={'CAT I'}, inplace=True)

In [15]:
#renaming revenue
train_df.rename(columns={"revenue ( $ Algérien)": "revenue"}, inplace=True)
test_df.rename(columns={"revenue ( $ Algérien)": "revenue"}, inplace=True)

#standardization revenue
train_df["revenue"] =( train_df["revenue"] - train_df["revenue"].mean() ) / train_df["revenue"].std()
test_df["revenue"] =( test_df["revenue"] - test_df["revenue"].mean() ) / test_df["revenue"].std()


In [ ]:
#label encoding other features 


le = LabelEncoder()
#train
le.fit(train_df['platform'])
cat_enc = le.transform(train_df["platform"])
train_df["platform"] = cat_enc

#test
cat_enc = le.transform(test_df["platform"])
test_df["platform"] = cat_enc
test_df


ord_enc_gender = LabelEncoder()
#train
ord_enc_gender.fit(train_df["gender"].unique())
cat_enc = ord_enc_gender.transform(train_df["gender"])
train_df["gender"] = cat_enc
#test
cat_enc = ord_enc_gender.transform(test_df["gender"])
test_df["gender"] = cat_enc
test_df



ord_enc_city = LabelEncoder()
#train
ord_enc_city.fit(train_df["city"].unique())
cat_enc = ord_enc_city.transform(train_df["city"])
train_df["city"] = cat_enc
#test
cat_enc = ord_enc_city.transform(test_df["city"])
test_df["city"] = cat_enc
test_df


In [17]:
#dropping useless columns
train_df.drop(['Order date'], axis=1, inplace=True)
train_df.drop(['id_purchase'], axis=1, inplace=True)
# train_df.drop(['platform'], axis=1, inplace=True)

test_df.drop(['Order date'], axis=1, inplace=True)
test_df.drop(['id_purchase'], axis=1, inplace=True)
# test_df.drop(['platform'], axis=1, inplace=True)


In [22]:
#remapping months
map_dict = {7:1 , 8:2, 9:3, 10:4, 11:5, 12:6 , 1:7}
train_df["month"] = train_df["month"].map(map_dict)
test_df["month"] = test_df["month"].map(map_dict)

In [21]:
test_df= test_df.drop(test_df.tail(7).index)

## **Grouping data**

In [23]:
# treating the platform frequency 
#this function replaces the column platform in df with the most frequent pplatform for a given customer 
def replace_platform(df):
    for id in df['Cust_id'].unique():
        cust_df = df[df['Cust_id'] == id]
        most_common_platform = cust_df['platform'].mode()[0]
        df.loc[df['Cust_id'] == id, 'platform'] = most_common_platform
    return df

test_df = replace_platform(test_df)
train_df = replace_platform(train_df)

In [24]:
# treating the city frequency 
#this function replaces the column city in df with the most frequent city for a given customer 
def replace_city(df):
    for id in df['Cust_id'].unique():
        cust_df = df[df['Cust_id'] == id]
        most_common_city = cust_df['city'].mode()[0]
        df.loc[df['Cust_id'] == id, 'city'] = most_common_city
    return df

test_df = replace_city(test_df)
train_df = replace_city(train_df)

In [25]:
#grouping
train_sum = train_df.groupby('Cust_id',as_index=False).max()
test_sum = test_df.groupby('Cust_id',as_index=False).max()

In [26]:
#dealing with conflicts after grouping
#train

avg_revenue_ = train_df.groupby(['Cust_id'],as_index=False)[['revenue']].mean()
max_gender_ = train_df.groupby(['Cust_id'],as_index=False)[['gender']].max()
max_city_ = train_df.groupby(['Cust_id'],as_index=False)[['city']].max()
max_cust_id_ = train_df.groupby(['Cust_id'],as_index=False)[['Cust_id']].max()
max_platform= train_df.groupby(['Cust_id'],as_index=False)[['platform']].max()
max_month = train_df.groupby(['Cust_id'],as_index=False)[['month']].max()


train_sum["revenue"] = avg_revenue_["revenue"]
train_sum["gender"] = max_gender_["gender"]
train_sum["Cust_id"] = max_cust_id_["Cust_id"]
train_sum["city"] = max_city_["city"]
train_sum["platform"]= max_platform["platform"]
train_sum["month"]= max_month["month"]

#test
avg_revenue = test_df.groupby(['Cust_id'],as_index=False)[['revenue']].mean()
max_gender= test_df.groupby(['Cust_id'],as_index=False)[['gender']].max()
max_city= test_df.groupby(['Cust_id'],as_index=False)[['city']].max()
max_cust_id= test_df.groupby(['Cust_id'],as_index=False)[['Cust_id']].max()
max_platform = test_df.groupby(['Cust_id'],as_index=False)[['platform']].max()
max_month = test_df.groupby(['Cust_id'],as_index=False)[['month']].max()

test_sum["revenue"] = avg_revenue["revenue"]
test_sum["gender"] = max_gender["gender"]
test_sum["Cust_id"] = max_cust_id["Cust_id"]
test_sum["city"] = max_city["city"]
test_sum["platform"]= max_platform["platform"]
test_sum["month"]= max_month["month"]

In [ ]:
#data augmentation
train_sum= pd.concat([train_sum, test_sum])
train_sum

# **Training**

In [ ]:
X_test = test_sum.iloc[:, :6]
X_test

In [ ]:
X = train_sum.iloc[:, 1:6]
Y = train_sum.iloc[:, 6:]
list(Y.values)

In [29]:
Y = np.array([ list(y) for y in Y.values ])

In [30]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score



# Split data into training and testing sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)


# Create and train a Random Forest classifier using OneVsRestClassifier
rfc = OneVsRestClassifier(RandomForestClassifier(n_estimators=400))
rfc.fit(X_train, Y_train)

# Make predictions on the test set
Y_pred = rfc.predict(X_val)

# Evaluate the model using accuracy, F1-score, and AUROC metrics
print("Accuracy:", accuracy_score(Y_val, Y_pred))
print("F1-score:", f1_score(Y_val, Y_pred, average='micro'))
print("AUROC:", roc_auc_score(Y_val, Y_pred, average='micro'))


In [ ]:
Y_pred = rfc.predict(X_test.iloc[:,1:])

## **Sample pre-processing** 

In [ ]:
sample_df = pd.read_csv("/content/sample_submission.csv")
# sample_df.rename(columns={"id":"Cust_id"}, inplace = True)
sample_df.drop(['cat' ], axis=1, inplace=True)
sample_df[['Cust_id', 'CAT II']] = sample_df['id'].str.split('_', expand=True)
sample_df['Cust_id'] = sample_df['Cust_id'].astype(int)
sample_df.drop(columns={"id"},inplace=True)


In [ ]:
sample_df

,Cust_id,CAT II
0,209886,Carrosserie
1,318591,Sacs
2,220722,Polos
3,320001,Outils de nettoyage
4,275099,Talons
...,...,...
2834,174894,Livres en langues étrangères
2835,290273,Grils et cuisine en plein air
2836,269582,Chaussures de sport
2837,323819,Couche


In [ ]:

output = []
for i , row in sample_df.iterrows():
   id = sample_df.loc[i]['Cust_id']
   index = list(X_test['Cust_id']).index(id)
   y = Y_pred[index]
   category = categories.index(row['CAT II'])
   if y[category] :
     output.append("buy")
   else:
     output.append("not buy")

 

In [ ]:
sample_df

,Cust_id,CAT II
0,209886,Carrosserie
1,318591,Sacs
2,220722,Polos
3,320001,Outils de nettoyage
4,275099,Talons
...,...,...
2834,174894,Livres en langues étrangères
2835,290273,Grils et cuisine en plein air
2836,269582,Chaussures de sport
2837,323819,Couche


# **Re-arranging submission sample**

In [ ]:
sample_df['Cust_id'] = sample_df['Cust_id'].astype(str)+ '_' + sample_df['CAT II']


In [ ]:
sample_df.rename(columns={'Cust_id': "Id"}, inplace=True)
sample_df.drop(['CAT II'], axis=1, inplace=True)
sample_df

,Id
0,209886_Carrosserie
1,318591_Sacs
2,220722_Polos
3,320001_Outils de nettoyage
4,275099_Talons
...,...
2834,174894_Livres en langues étrangères
2835,290273_Grils et cuisine en plein air
2836,269582_Chaussures de sport
2837,323819_Couche


In [ ]:
sample_df['cat'] = [i for i in output]
sample_df

,Id,cat
0,209886_Carrosserie,not buy
1,318591_Sacs,not buy
2,220722_Polos,not buy
3,320001_Outils de nettoyage,not buy
4,275099_Talons,not buy
...,...,...
2834,174894_Livres en langues étrangères,not buy
2835,290273_Grils et cuisine en plein air,buy
2836,269582_Chaussures de sport,not buy
2837,323819_Couche,buy


In [ ]:
sample_df.to_csv('submission.csv', index=False)